# Test out Hyperparameter tuning


In [1]:
from venus_ml import HyperparameterTuner, RandomForest, VenusDataset, BaseMLP
from torch.optim import SGD
from torch.nn import MSELoss, L1Loss
import numpy as np

### Get dataset

In [2]:
data_set_config = {
    "file_path": "accumulated_weekend_data_2023_02_27.parquet",
    "input_columns": ["inj_mbar_mean", "bias_v_mean", "bias_i_mean", "extraction_i_mean"],
    "output_columns": ["fcv1_i_mean"],
    "run_selection":5
}
dataset = VenusDataset(**data_set_config)
X, y = dataset.to_tensor()
# y = y.ravel()
print(X.shape, y.shape)
print(np.count_nonzero(np.isnan(X)))

torch.Size([3556, 4]) torch.Size([3556, 1])
0


### Set up sklearn model

In [3]:
hyps = {
    "n_estimators": 100
}
sk_model = RandomForest(hyps)
sk_param_grid = {
    "n_estimators": [10, 25, 100],
    "criterion": ["squared_error", "absolute_error", "friedman_mse"],
    "bootstrap": [True, False]
}

### Set up simple pytorch model

In [4]:
hyps = {
    "layer_sizes": [4, 8, 8, 1]
}
optimizer_params = {
    "lr": 1e-3
}
torch_model = BaseMLP(hyps, optimizer_params, SGD, MSELoss)
torch_param_grid = {
    "lr": [1e-1, 1e-2, 1e-3, 1e-4, 1e-5],
    "max_epochs": [1, 20],
}

### Initialize Tuners

In [5]:
sk_tuner = HyperparameterTuner(sk_model, sk_param_grid, verbosity=3)
torch_tuner = HyperparameterTuner(torch_model, torch_param_grid, verbosity=3)

### Run Sklearn Tuner

In [6]:
sk_y = y.ravel()
sk_tuner.tune(X, sk_y)

Fitting 5 folds for each of 18 candidates, totalling 90 fits
[CV 1/5] END bootstrap=True, criterion=squared_error, n_estimators=10;, score=0.984 total time=   0.0s
[CV 2/5] END bootstrap=True, criterion=squared_error, n_estimators=10;, score=0.985 total time=   0.0s
[CV 3/5] END bootstrap=True, criterion=squared_error, n_estimators=10;, score=0.983 total time=   0.0s
[CV 4/5] END bootstrap=True, criterion=squared_error, n_estimators=10;, score=0.979 total time=   0.0s
[CV 5/5] END bootstrap=True, criterion=squared_error, n_estimators=10;, score=0.974 total time=   0.0s
[CV 1/5] END bootstrap=True, criterion=squared_error, n_estimators=25;, score=0.984 total time=   0.1s
[CV 2/5] END bootstrap=True, criterion=squared_error, n_estimators=25;, score=0.986 total time=   0.1s
[CV 3/5] END bootstrap=True, criterion=squared_error, n_estimators=25;, score=0.983 total time=   0.1s
[CV 4/5] END bootstrap=True, criterion=squared_error, n_estimators=25;, score=0.980 total time=   0.1s
[CV 5/5] END

### Run Torch Tuner

In [7]:
torch_tuner.tune(X, y)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
  epoch    train_loss    valid_loss     dur
-------  ------------  ------------  ------
      1   328932.3879      464.2746  0.0258
[CV 1/5] END ...lr=0.1, max_epochs=1;, score=-25088007635.199 total time=   0.0s
  epoch    train_loss    valid_loss     dur
-------  ------------  ------------  ------
      1   359638.2430      513.0765  0.0077
[CV 2/5] END ...lr=0.1, max_epochs=1;, score=-27065733361.423 total time=   0.0s
  epoch    train_loss    valid_loss     dur
-------  ------------  ------------  ------
      1   227175.8920      305.4815  0.0078
[CV 3/5] END ...lr=0.1, max_epochs=1;, score=-16315718486.020 total time=   0.0s
  epoch    train_loss    valid_loss     dur
-------  ------------  ------------  ------
      1   227175.8905      305.4815  0.0080
[CV 4/5] END ...lr=0.1, max_epochs=1;, score=-18365954269.810 total time=   0.0s
  epoch    train_loss    valid_loss     dur
-------  ------------  ------------  ------